In [2]:
from vidHandlr import *
import numpy as np
import cv2, time
import pickle as pik
import pandas as pd
import ezodf, csv, re, os

In [3]:
#Location of file with list of all videos and locations
datasheet = '/home/talcat/VTDrive/Talia Weiss/Projects/Cricket Frog Skittering/Analyses/CricketFrogPaper 052018/AboveBelow.csv'

In [3]:
sheet = pd.read_csv(datasheet, encoding='utf-16')
sheet

,FROG,Sequence,ID,file location,Unnamed: 4,FPS,# Launch,# Land
0,AC01,1,AC01_01,/media/talcat/data/Acris spp/2013_10_31/2013-1...,NaN,500,2,1
1,NaN,2,AC01_02,/media/talcat/data/Acris spp/2013_10_31/2013-1...,NaN,500,3,2
2,NaN,3,AC01_03,/media/talcat/data/Acris spp/2013_10_31/2013-1...,NaN,500,1,1
3,NaN,4,AC01_04,/media/talcat/data/Acris spp/2013_10_31/2013-1...,NaN,500,1,1
4,NaN,5,AC01_05,/media/talcat/data/Acris spp/2013_10_31/2013-1...,NaN,500,2,2
5,NaN,6,AC01_06,/media/talcat/data/Acris spp/2013_10_31/2013-1...,NaN,500,2,2
6,NaN,7,AC01_07,/media/talcat/data/Acris spp/2013_10_31/2013-1...,NaN,500,1,0
7,NaN,8,AC01_08,/media/talcat/data/Acris spp/2013_10_31/2013-1...,NaN,250,1,0
8,NaN,9,AC01_09,/media/talcat/data/Acris spp/2013_10_31/2013-1...,NaN,250,2,1
9,NaN,10,AC01_10,/media/talcat/data/Acris spp/2013_10_31/2013-1...,NaN,250,3,2


In [4]:
k = sheet.loc[sheet['ID'] == 'AC01_01']['file location']

In [5]:
k.item()

'/media/talcat/data/Acris spp/2013_10_31/2013-10-31 AC 01 01_000000'

In [15]:
def aboveOrBelow(ID):  
        global sheet
        
        imdir = sheet.loc[sheet['ID'] == ID]['file location']
        print(imdir)
        vid = vidHandlr(imdir.item())

        filename = ID
    
        date = time.strftime('%Y%m%d')
    
        filenamefull = 'DataAnalysis/AboveBelow/%s-%s.csv' %(filename,date)
        
        state = ['-']*vid.n_frames
                 
        
                    
        cv2.namedWindow('img', cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_NORMAL )
        
        #brightness, contrast?
        cv2.createTrackbar("gamma", 'img', 0, 255, lambda x:x )
        cv2.setTrackbarMin('gamma', 'img', 0)
          
        framelist = list(range(vid.n_frames))
        framelist.sort()
        curIdx = 0
        
        
        def getAdjustIm(gamma, im):
             
            
        
            #get HSV
            hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
        
            hsv = hsv.astype(np.float64)
        
            hsv[:,:,2]= hsv[:,:,2] * (10*gamma/(255) + 1.0)
        
            themax = np.max(hsv)
            #print(themax)
        
            hsv[hsv > 255] = 255
        
            hsv = hsv.astype('uint8')
        
            im2 = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
            return im2.astype(np.uint8)
        
        
        
        
        
        
        running = True
        while running:
            thekey = framelist[curIdx]
            
            image = vid.get_frame(int(thekey), 'BGR')
            gamma = cv2.getTrackbarPos('gamma', 'img')
            
            im = getAdjustIm(gamma, image)
            
            cv2.imshow('img', im)
            
            curstate = ''
            
            curstate = '%s, ' %(state[curIdx])

            cv2.displayOverlay('img', "Frame %d/%d \nState %s" %(curIdx, vid.n_frames, curstate), 0)

            
            k = cv2.waitKey(10)
            if k == 27: #Escape
                cv2.destroyAllWindows()
                running=False
                break
            elif k in [65363, ord('c'), ord('C')]: #right arrow
                if curIdx + 1 < len(framelist):
                    curIdx += 1
            elif k in [65361, ord('z'), ord('Z')]: #left arrow
                if curIdx - 1 > 0:
                    curIdx -= 1
            elif k in [ord('a'), ord('A')]: #above water
                state[curIdx] = 'a'
                curIdx +=1
                
            elif k in [ord('d'), ord('D')]: #water contact
                state[curIdx] = 'c'
                curIdx +=1
                    
            elif k in [ord('b'), ord('B')]: #below water
                state[curIdx] = 'b'
                curIdx +=1
                
            if curIdx + 1> vid.n_frames:
                running = False
                curIdx -= 1
                cv2.destroyAllWindows()
                break
                
                
                

        with open(filenamefull, 'w') as f:
            f.write('Frameno, state\n')
            for fr in range(vid.n_frames):
                f.write('%d, %s\n' %(fr, str(state[fr])))
        
        return state

In [4]:
seqs = os.listdir('DataAnalysis/AboveBelow')
seqs.sort()
print(seqs)

['AC01_01-20180725.csv', 'AC01_02-20180725.csv', 'AC01_03-20180725.csv', 'AC01_04-20180725.csv', 'AC01_05-20180725.csv', 'AC01_06-20180725.csv', 'AC01_07-20180725.csv', 'AC01_08-20180725.csv', 'AC01_09-20180725.csv', 'AC01_10-20180725.csv', 'AC01_11-20180725.csv', 'AC02_01-20180725.csv', 'AC02_02-20180725.csv', 'AC02_03-20180725.csv', 'AC02_04-20180725.csv', 'AC02_05-20180725.csv', 'AC03_01-20180725.csv', 'AC03_02-20180725.csv', 'AC03_03-20180725.csv', 'AC03_04-20180725.csv', 'AC03_05-20180725.csv', 'AC03_06-20180725.csv', 'AC03_07-20180725.csv', 'AC03_08-20180725.csv', 'AC03_09-20180725.csv', 'AC03_10-20180725.csv', 'AC03_11-20180725.csv', 'AC03_12-20180725.csv', 'AC03_13-20180725.csv', 'AC03_14-20180725.csv', 'AC03_15-20180725.csv', 'AC03_16-20180725.csv', 'AC03_17-20180725.csv', 'AC03_18-20180725.csv', 'AC03_19-20180725.csv', 'AC03_20-20180725.csv', 'AC03_21-20180725.csv', 'AC03_22-20180725.csv', 'AC03_23-20180725.csv', 'AC03_24-20180725.csv', 'AC04_01-20180725.csv', 'AC04_02-201807

In [60]:
curseq = pd.read_csv('DataAnalysis/AboveBelow/%s' %seqs[0])

In [71]:
k = curseq[' state'].values

In [77]:
''.join(k).replace(' ', '')

'---------------------------------------------------------------------------------------bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbcccccccccccccaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaacccccccccccbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbccccccccccaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa--------'

In [8]:
for seq in seqs:
    cur_seq = pd.read_csv('DataAnalysis/AboveBelow/%s' %seq)
    vals = cur_seq[' state'].values
    string = ''.join(vals).replace(' ', '')
    print(seq)
    for th in re.finditer(r'bc+a+c+b+c', string):
        st, en = th.span()
        substr = string[st:en]
        
        #Launch transition
        lt = re.search(r'bc+a', substr)
        lts, lte = lt.span()
        print('Launch transition %d' %(lte-lts))
        
        #Above the surface
        above = re.search(r'ca+c', substr)
        ast, aen = above.span()
        print('Above %d' %(aen - ast))
        
        #Land Transition
        lt = re.search(r'ac+b', substr)
        lts, lte = lt.span()        
        print('Land transition %d' %(lte-lts))       
        
        #Below surface
        below = re.search(r'cb+c', substr)
        bst, ben = below.span()
        print('Below %d' %(ben- bst))
        print('\n')
        

AC01_01-20180725.csv
Launch transition 15
Above 56
Land transition 13
Below 60


AC01_02-20180725.csv
Launch transition 13
Above 77
Land transition 12
Below 45


AC01_03-20180725.csv
AC01_04-20180725.csv
AC01_05-20180725.csv
Launch transition 11
Above 89
Land transition 14
Below 46


AC01_06-20180725.csv
AC01_07-20180725.csv
AC01_08-20180725.csv
AC01_09-20180725.csv
Launch transition 6
Above 52
Land transition 8
Below 28


AC01_10-20180725.csv
Launch transition 9
Above 42
Land transition 8
Below 39


AC01_11-20180725.csv
AC02_01-20180725.csv
AC02_02-20180725.csv
AC02_03-20180725.csv
Launch transition 6
Above 36
Land transition 9
Below 29


AC02_04-20180725.csv
Launch transition 10
Above 48
Land transition 8
Below 30


AC02_05-20180725.csv
Launch transition 13
Above 6
Land transition 13
Below 39


AC03_01-20180725.csv
AC03_02-20180725.csv
AC03_03-20180725.csv
Launch transition 7
Above 64
Land transition 6
Below 31


AC03_04-20180725.csv
Launch transition 6
Above 43
Land transition 8
Bel